In [1]:
import os
import PIL.Image as piImg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mp
import shutil

In [11]:
url = 'C:/Users/M/Downloads/153.반려동물 안구질환 데이터/01.데이터/1.Training/라벨링데이터/TL2/고양이/안구/일반/cat_eye_normal'

In [40]:
unormal_dir = os.path.join(url, 'unormal')
normal_dir= os.path.join(url, 'normal')
train_dir = os.path.join(url, 'train')
test_dir = os.path.join(url, 'test')
val_dir = os.path.join(url, 'val')
os.mkdir(unormal_dir)
os.mkdir(train_dir)
os.mkdir(test_dir)
os.mkdir(val_dir)

In [19]:
train_unormal_dir = os.path.join(train_dir, 'unormal')
train_normal_dir = os.path.join(train_dir, 'normal')

test_unormal_dir = os.path.join(test_dir, 'unormal')
test_normal_dir = os.path.join(test_dir, 'normal')

val_unormal_dir = os.path.join(val_dir, 'unormal')
val_normal_dir = os.path.join(val_dir, 'normal')

In [20]:
os.mkdir(train_unormal_dir)
os.mkdir(train_normal_dir)

os.mkdir(test_unormal_dir)
os.mkdir(test_normal_dir)

os.mkdir(val_unormal_dir)
os.mkdir(val_normal_dir)

In [23]:
corneal_ulcer_file = os.listdir(url + '/corneal_ulcer')[:500]
corneal_flap_file = os.listdir(url + '/corneal_flap')[:500]
conjunctivitis_file = os.listdir(url + '/conjunctivitis')[:500]
keratitis_file = os.listdir(url + '/keratitis')[:500]
blepharitis_file = os.listdir(url + '/blepharitis')[:500]


In [34]:
for f in corneal_ulcer_file:
    shutil.copy(url + '/corneal_ulcer/' + f, unormal_dir)
for f in corneal_flap_file:
    shutil.copy(url + '/corneal_flap/' + f, unormal_dir)
for f in conjunctivitis_file:
    shutil.copy(url + '/conjunctivitis/' + f, unormal_dir)
for f in keratitis_file:
    shutil.copy(url + '/keratitis/' + f, unormal_dir)
for f in blepharitis_file:
    shutil.copy(url + '/blepharitis/' + f, unormal_dir)

In [37]:
normal_file = os.listdir(url + '/normal')[:2500]
unormal_file = os.listdir(url + '/unormal')[:2500]

In [36]:
len(unormal_file)

2500

In [39]:
print(len(normal_file),len(unormal_file))

2500 2500


In [42]:
for f in normal_file[:2000]:
    shutil.copy(url + '/normal/' + f, train_normal_dir)
for f in unormal_file[:2000]:
    shutil.copy(url + '/unormal/' + f, train_unormal_dir)

for f in normal_file[2000:2250]:
    shutil.copy(url + '/normal/' + f, test_normal_dir)
for f in unormal_file[2000:2250]:
    shutil.copy(url + '/unormal/' + f, test_unormal_dir)

for f in normal_file[2250:]:
    shutil.copy(url + '/normal/' + f, val_normal_dir)
for f in unormal_file[2250:]:
    shutil.copy(url + '/unormal/' + f, val_unormal_dir)

In [43]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,Model

In [44]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [45]:
train_data_importer = train_datagen.flow_from_directory(
    url+'/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_data_importer = train_datagen.flow_from_directory(
    url+'/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_data_importer = train_datagen.flow_from_directory(
    url+'/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 5000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [50]:
import tensorflow as tf
from tensorflow.keras import layers, models

def identity_block(input_tensor, filters, kernel_size):
    filters1, filters2, filters3 = filters

    x = layers.Conv2D(filters1, (1, 1), kernel_initializer='he_normal')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

def conv_block(input_tensor, filters, kernel_size, strides=(2, 2)):
    filters1, filters2, filters3 = filters

    x = layers.Conv2D(filters1, (1, 1), strides=strides, kernel_initializer='he_normal')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, (1, 1), kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    shortcut = layers.Conv2D(filters3, (1, 1), strides=strides, kernel_initializer='he_normal')(input_tensor)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def resnet50():
    input = layers.Input(shape=(224, 224, 3))
    x = layers.ZeroPadding2D((3, 3))(input)
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = conv_block(x, [64, 64, 256], (3, 3), strides=(1, 1))
    x = identity_block(x, [64, 64, 256], (3, 3))
    x = identity_block(x, [64, 64, 256], (3, 3))

    x = conv_block(x, [128, 128, 512], (3, 3))
    x = identity_block(x, [128, 128, 512], (3, 3))
    x = identity_block(x, [128, 128, 512], (3, 3))
    x = identity_block(x, [128, 128, 512], (3, 3))

    x = conv_block(x, [256, 256, 1024], (3, 3))
    x = identity_block(x, [256, 256, 1024], (3, 3))
    x = identity_block(x, [256, 256, 1024], (3, 3))
    x = identity_block(x, [256, 256, 1024], (3, 3))
    x = identity_block(x, [256, 256, 1024], (3, 3))
    x = identity_block(x, [256, 256, 1024], (3, 3))

    x = conv_block(x, [512, 512, 2048], (3, 3))
    x = identity_block(x, [512, 512, 2048], (3, 3))
    x = identity_block(x, [512, 512, 2048], (3, 3))

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(2, activation='sigmoid')(x)

    model = models.Model(inputs=input, outputs=x)
    return model

model = resnet50()
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_2    │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_106 (Conv2D) │ (None, 112, 112,  │      9,472 │ zero_padding2d_2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_106[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 56, 56,    │          0 │ activation_98[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_107 (Conv2D) │ (None, 56, 56,    │      4,160 │ max_pooling2d_2[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_107[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_99       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_108 (Conv2D) │ (None, 56, 56,    │     36,928 │ activation_99[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_108[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_100      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_109 (Conv2D) │ (None, 56, 56,    │     16,640 │ activation_100[0… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_110 (Conv2D) │ (None, 56, 56,    │     16,640 │ max_pooling2d_2[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_109[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_110[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_32 (Add)        │ (None, 56, 56,    │          0 │ batch_normalizat

 Total params: 23,591,810 (90.00 MB)

 Trainable params: 23,538,690 (89.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [51]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [52]:
hist = model.fit(
    train_data_importer,
    epochs=5,
    validation_data=val_data_importer
)

Epoch 1/5


c:\Users\M\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


157/157 ━━━━━━━━━━━━━━━━━━━━ 1276s 8s/step - accuracy: 0.6525 - loss: 0.7740 - val_accuracy: 0.6610 - val_loss: 1.0727
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8148 - loss: 0.4249

KeyboardInterrupt: 